In [1]:
import sys
sys.path.append("./Bspli/")
import os  
import faiss
import time
import numpy as np 
import index 
import torch
import pandas as pd

mnist = np.load("dataset/mnist-784-euclidean.npy")
print(f'mnist data shape: {mnist.shape}')
print(f'mnist dtype: {mnist.dtype}')

# mnist = np.random.rand(10000, 784)
# mnist = mnist.astype(np.float32)
# print(f'mnist dtype: {mnist.dtype}')
# print(f'mnist data shape: {mnist.shape}')


mnist dtype: float32
mnist data shape: (10000, 784)


In [2]:
%%time
# Brute Search
flat = faiss.IndexFlatL2(mnist.shape[1])
flat.add(mnist)

D, FLAT_I = flat.search(mnist[0].reshape(1, mnist.shape[1]), k=100) 
print(f'brute query: {FLAT_I}')

brute query: [[   0   97   92  930 7294 5918 5574 7534 7898 7353 7805 4977 2402 4335
  5740  742 1694 7398 6924 1809 7768 1765 1705 7050 9970 1563 4375 8846
  4146 9821 8130 4459 9441 6375 8178 9050 9344   98 3419 6708 6755 8030
  3356 5395 5786 5818 6009 8861 7201 2588 4152 4261 4102 1271 1134 9227
  2285 2659 3237 4980 5414 8104 2515 7543 1966 3828  728  164  589 7941
   802 4163  855 2850 7735  873 3057 9772 3865 1536 8209 6874 4230 9466
  1920 7733 5842 6583 6016  615 6149  493 9678 6123 2618 7143 3463 5479
  4507  467]]
CPU times: total: 0 ns
Wall time: 13 ms


In [3]:
mnist_tensor = torch.from_numpy(mnist)
print(f'mnist tensor shape: {mnist_tensor.shape}')

idx = index.Indexing(
    gl_size=80000, 
    ll_size=1000,
    g_epoch_num=5,
    l_epoch_num=20,
    g_hidden_size=5,
    l_hidden_size=10,
    l_block_range=60,
    random_partitioning=False
)
idx.train(mnist_tensor)

print(f"local model len:{len(idx._l_model)}")

mnist tensor shape: torch.Size([10000, 784])
torch.Size([10000, 785])
first stage partitioning finish
partitioning blocks : 1
training local model
trainging global model
global index train smaple count: 17
finish
local model len:1


d:\study\bspli\./Bspli\utils\Partitioning.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ma = torch.tensor(means)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


In [4]:
%%time

def recall(pred, true):
    x = np.isin(pred, true)
    return x.sum() / true.size

qp = torch.from_numpy(mnist[0])
# print(qp)
pred = idx.query(qp, k=100)
print(f"recall: {recall(pred, FLAT_I)}")

pred = pred.to(torch.int)
print(f"pred: {pred}")

recall: 0.83
pred: tensor([   0,   97,  930, 7294, 5918, 7534, 7898, 7353, 7805, 4977, 4335, 5740,
         742, 1694, 7398, 6924, 1809, 1765, 1705, 9970, 1563, 4375, 8846, 4146,
        9821, 8130, 4459, 9441, 6375, 9050, 9344,   98, 3419, 6708, 8030, 3356,
        5395, 5786, 5818, 6009, 8861, 7201, 2588, 4152, 4261, 4102, 1271, 9227,
        2285, 2659, 4980, 8104, 2515, 7543, 1966, 3828,  728, 7941,  802, 4163,
         855, 2850, 7735,  873, 3865, 1536, 6874, 9466, 1920, 7733, 5842, 6583,
        6016,  615, 6149,  493, 9678, 6123, 2618, 7143, 5479, 4507,  467, 7028,
        4932, 4563,  189, 6093, 3323, 3667,  653, 5229, 8745, 8192, 9786, 9055,
        7001, 4297, 2763,  645], dtype=torch.int32)
CPU times: total: 62.5 ms
Wall time: 16 ms


In [5]:
result = []

def benchmark_knn_query(data, index, size=1000, k=100):
    indices = np.random.choice(data.shape[0], size, replace=False)
    query_time = 0
    cur_recall = 0

    # query
    for i in indices:
        q = torch.from_numpy(data[i])
        start = time.time()
        qk = index.query(q, k=100)
        query_time += (time.time() - start)
        D, FLAT_I = flat.search(data[i].reshape(1, data.shape[1]), k=k) 
        cur_recall += recall(qk, FLAT_I)
    result.append((query_time/1000, cur_recall/1000))

In [6]:
# recall(pred, FLAT_I)

benchmark_knn_query(mnist, idx, size=1000, k=100)

In [7]:
print(result)

[(0.012954625129699707, 0.8198499999999997)]
